# Proximal Policy Optimitization (PPO)

In [ ]:
!pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.5 MB/s eta 0:00:00


In [ ]:
!pip install stable_baselines3 gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 27.5 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376424 sha256=79c1f8b7df5a78f4bc914c2d1e27bd8e677925b1a881af7b7397601b09d7f73b
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.atari_wrappers import WarpFrame
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecTransposeImage
from stable_baselines3.common.vec_env import VecNormalize
import os
import torch
import numpy
import platform
import matplotlib
import matplotlib.pyplot
from platform import python_version
from importlib.metadata import version

In [ ]:
env = gym.make("CarRacing-v3", continuous=False)
print("Observation Space Size: ", env.observation_space.shape)
print("Action Space Size: ", env.action_space.shape)
env.close()

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


Observation Space Size:  (96, 96, 3)
Action Space Size:  ()


/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

In [ ]:
env_str = "CarRacing-v3"
log_dir = "./logs/{}".format(env_str)
gray_scale = True

# If gray_scale True, convert obs to gray scale 84 x 84 image
wrapper_class = WarpFrame if gray_scale else None

In [ ]:
# Create Training CarRacing environment
env = make_vec_env(env_str, n_envs=1, wrapper_class=wrapper_class)
env = VecFrameStack(env, n_stack=4)
env = VecTransposeImage(env)
#env = VecNormalize(env, norm_reward=True)

# Create Evaluation CarRacing environment
env_val = make_vec_env(env_str, n_envs=1, wrapper_class=wrapper_class)
env_val = VecFrameStack(env_val, n_stack=4)
env_val = VecTransposeImage(env_val)

# Create Evaluation Callback
# eval_freq - can cause learning instability if set to low
eval_callback = EvalCallback(env_val,
                             best_model_save_path=log_dir,
                             log_path=log_dir,
                             eval_freq=20_000, render=False,
                             n_eval_episodes=20)

# Initialize PPO
# ent_coef - encourages exploration of other actins
model = PPO('CnnPolicy', env, n_steps=10_000, verbose=1, ent_coef=0.005)

# Train the model
model.learn(total_timesteps=100_000, progress_bar=True, callback=eval_callback)

# Save the model
model.save(os.path.join(log_dir, "ppo_car_racing"))

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=20)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

env.close()
env_val.close()

Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 10000`, after every 156 untruncated mini-batches, there will be a truncated mini-batch of size 16
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=10000 and n_envs=1)
  warnings.warn(


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -53.5    |
| time/              |          |
|    fps             | 43       |
|    iterations      | 1        |
|    time_elapsed    | 231      |
|    total_timesteps | 10000    |
---------------------------------


Eval num_timesteps=20000, episode_reward=-13.59 +/- 18.37

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -59.7    |
| time/              |          |
|    fps             | 15       |
|    iterations      | 8        |
|    time_elapsed    | 5268     |
|    total_timesteps | 80000    |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1e+03     |
|    ep_rew_mean          | -60       |
| time/                   |           |
|    fps                  | 15        |
|    iterations           | 9         |
|    time_elapsed         | 5722      |
|    total_timesteps      | 90000     |
| train/                  |           |
|    approx_kl            | 0.0335256 |
|    clip_fraction        | 0.266     |
|    clip_range           | 0.2       |
|    entropy_loss         | -4.02     |
|    explained_variance   | 0.803     |
|    learning_rate        | 0.0003    |
|    loss           

Eval num_timesteps=100000, episode_reward=-23.34 +/- 38.16

Episode length: 986.25 +/- 51.31

----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 986        |
|    mean_reward          | -23.3      |
| time/                   |            |
|    total_timesteps      | 100000     |
| train/                  |            |
|    approx_kl            | 0.03353564 |
|    clip_fraction        | 0.282      |
|    clip_range           | 0.2        |
|    entropy_loss         | -3.98      |
|    explained_variance   | 0.887      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0758    |
|    n_updates            | 90         |
|    policy_gradient_loss | -0.0303    |
|    std                  | 0.911      |
|    value_loss           | 0.118      |
----------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -60.5    |
| time/              |          |
|    fps             | 15       |
|    iterations  

Mean reward: -41.62 +/- 50.80
